In [1]:
# ---- Step 1: Import Libraries ----
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical


# ---- Step 2: Load Dataset ----
df = pd.read_json("D:\\Downloads\\archive\\News_Category_Dataset_v3.json", lines=True)
df = df[['headline','category']]

# Filter categories
categories = ["POLITICS", "SPORTS", "BUSINESS", "TECH", "CRIME"]
df = df[df['category'].isin(categories)]

# Encode labels as numbers
le = LabelEncoder()
y = le.fit_transform(df['category'])
y_cat = to_categorical(y)   # one-hot encoding

X = df['headline']

# ---- Step 3: Train-test split ----
X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat, test_size=0.2, random_state=42, stratify=y_cat
)

# ---- Step 4: TF-IDF Vectorization ----
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()

# ---- Step 5: Build Neural Network ----
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train_tfidf.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(categories), activation='softmax'))  # output layer

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ---- Step 6: Train ----
history = model.fit(
    X_train_tfidf, y_train,
    epochs=5, batch_size=32,
    validation_split=0.2,
    verbose=1
)

# ---- Step 7: Evaluate ----
loss, acc = model.evaluate(X_test_tfidf, y_test, verbose=0)
print(f"\nTest Accuracy: {acc:.4f}")

# ---- Step 8: User Input Prediction ----
while True:
    user_input = input("Enter a news article (or 'quit'): ")
    if user_input.lower() == "quit":
        break
    
    user_tfidf = vectorizer.transform([user_input]).toarray()
    pred = model.predict(user_tfidf)
    pred_class = le.classes_[np.argmax(pred)]
    
    print("Predicted Category:", pred_class)
    print("-"*50)


ModuleNotFoundError: No module named 'tensorflow.python'